In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score
import seaborn as sns
from matplotlib import pyplot as plt
import regex as re
from sklearn.feature_extraction.text import CountVectorizer
import collections
from sklearn_pandas import DataFrameMapper
import scipy


In [2]:
def preprocessing(txt):
    txt = re.sub('[^a-zA-Z0-9 ]', '', txt)
    txt = txt.lower()
    return txt 

In [3]:
train_file = pd.read_csv('../data//train.csv')
final_test_file = pd.read_csv('../data/test.csv')

In [4]:
title1_en=train_file['title1_en'].apply(preprocessing)
title2_en = train_file['title2_en'].apply(preprocessing)
train_file['title1_en']=title1_en
train_file['title2_en']=title2_en

title1_en=final_test_file['title1_en'].apply(preprocessing)
title2_en = final_test_file['title2_en'].apply(preprocessing)
final_test_file['title1_en']=title1_en
final_test_file['title2_en']=title2_en

In [5]:
vectorizer = TfidfVectorizer(stop_words="english", strip_accents="unicode", lowercase=False,encoding='utf-8', min_df=30).fit(train_file["title1_en"]+" "+train_file["title2_en"])


In [6]:
train_title1=vectorizer.transform( train_file['title1_en'])
train_title2=vectorizer.transform( train_file['title2_en'])
# train_file['title1_en']=vectorizer.transform( train_file['title1_en']).todense()
# train_file['title2_en']=vectorizer.transform( train_file['title2_en']).todense()
# print(train_title1.shape)
# print(train_title2.shape)
# print(type(train_title1))
# print(type(train_title2))


In [12]:
print(train_title1.shape)
print(train_title2.shape)
print(scipy.sparse.hstack([train_title1,train_title2]).shape)

train_features = scipy.sparse.hstack([train_title1,train_title2]).todense()
print(train_features.shape)

(256442, 10893)
(256442, 10893)
(256442, 21786)
(256442, 21786)


In [15]:
print(train_features[0].shape)

(1, 21786)


In [16]:
%%time
logistic_regression_model = LogisticRegression(penalty="l2", solver="lbfgs", multi_class="multinomial", max_iter=200, C=0.9).fit(train_features, train_file['label'])




C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [7]:
train_split, test_split = train_test_split(train_file, test_size=0.3)


In [8]:
Y_label_test = test_split.label
Y_label_train = train_split.label

In [9]:
train_split.shape

(179509, 6)

In [10]:
X_train=(train_split[['title1_en','title2_en']])
X_test = (test_split[['title1_en', 'title2_en']])
X_test_final = final_test_file[['title1_en', 'title2_en']]


In [11]:
print(type(X_train['title1_en']))
print(len(X_train['title1_en']))
print(len(X_train['title2_en']))
print(len(pd.concat([X_train['title1_en'], X_train['title2_en']])))

<class 'pandas.core.series.Series'>
179509
179509
359018


In [12]:
train_df=pd.concat([X_train['title1_en'], X_train['title2_en']])
words=set()
for i in train_df.array:
    
    string=i.split()
    for s in string:
        words.add(s)
len(words)


46821

In [13]:

from sklearn.compose import ColumnTransformer
# vectorizer = TfidfVectorizer(stop_words="english", strip_accents="unicode", lowercase=False,
#                              # pd.concat([X_train['title1_en'], X_train['title2_en']]
#                              encoding='utf-8', min_df=9).fit(pd.concat([X_train['title1_en'], X_train['title2_en']]))
# ct = ColumnTransformer(
#     [("title1_en", (vectorizer)), ("title2_en", (vectorizer))], remainder='passthrough')
# print(zip(*ct.transformers))
map_v = DataFrameMapper([
    # TfidfVectorizer(lowercase=False, encoding='utf-8',min_df=10)
    ('title1_en', vectorizer),
    ('title2_en', vectorizer)
])

# title_feat =ct.fit_transform(X_train)
# title_feat = map_v.fit_transform(X_train)

# # X_train = vectorizer.transform(train_split[['title1_en', 'title2_en']])
# # X_test = vectorizer.transform(test_split[['title1_en','title2_en']])

# X_train_col_1 = vectorizer.transform(X_train['title1_en'])
# X_train_col_2 = vectorizer.transform(X_train['title2_en'])

# X_test_col_1 = vectorizer.transform(X_test['title1_en'])
# X_test_col_2= vectorizer.transform(X_test['title2_en'])

# X_test_final_col_1 = vectorizer.transform(X_test_final['title1_en'])
# X_test_final_col_2= vectorizer.transform(X_test_final['title2_en'])


In [14]:
title_feat = map_v.fit_transform(X_train)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
print(title_feat.shape)

(179509, 12399)


In [16]:
# print(X_train_col_1.shape)
# print(X_train_col_2.shape)

# print(X_test_col_1.shape)
# print(X_test_col_2.shape)

# print(X_test_final_col_1.shape)
# print(X_test_final_col_2.shape)


In [17]:
# X_train = pd.concat([list(X_train_col_1), list(X_train_col_2)], axis=1)
# print(X_train.shape)

# X_test = pd.concat([list(X_test_col_1), list(X_test_col_2)], axis=1)
# print(X_test.shape)

# X_test_final=pd.concat([list(X_test_final_col_1),list(X_test_final_col_2)],axis=1)
# print(X_test_final.shape)

In [18]:
# print(Y_label_test.shape)
# print(Y_label_train.shape)
# print(X_train.shape)
# print(X_test.shape)


feature engineering?

In [ ]:
%%time
logistic_regression_model = LogisticRegression(penalty="l2",
                               solver="lbfgs",
                               multi_class="multinomial",
                                               max_iter=200, C=0.9).fit(title_feat, Y_label_train)




In [ ]:
logistic_regression_model.score(title_feat, Y_label_train)


0.8449938443197834

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(logistic_regression_model, open(filename, 'wb'))


In [25]:
map_t = DataFrameMapper([
    # TfidfVectorizer(lowercase=False, encoding='utf-8',min_df=10)
    ('title1_en', vectorizer),
    ('title2_en', vectorizer)
])

In [26]:
test_mod = map_t.transform(X_test)
test_mod.shape

AttributeError: 'DataFrameMapper' object has no attribute 'built_features'

In [ ]:
import pickle
filename = 'finalized_model.sav'

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(test_mod, Y_label_test)
print(result)


ValueError: X has 15812 features, but LogisticRegression is expecting 24619 features as input.

In [ ]:
Y_hat_test = logistic_regression_model.predict(test_mod)


In [ ]:
print(classification_report(Y_label_test, Y_hat_test))


In [ ]:
cnf_matrix_l1_primary_dataset = metrics.confusion_matrix(Y_label_test, Y_hat_test)
%matplotlib inline
class_names = ["agreed", "disagreed","unrelated"]  # name  of classes
# fig, ax = plt.subplots()
plt.figure(figsize=(10, 10))
tick_marks = np.arange(len(class_names))
print(tick_marks)
plt.xticks([0,1,2], class_names)
plt.yticks([0,1,2],class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix_l1_primary_dataset), annot=True,cmap="YlGnBu", fmt='g', xticklabels=class_names, yticklabels=class_names)
plt.tight_layout()
plt.title('Confusion matrix for Primary test dataset Logistic regression with penalty', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

plt.show()


In [ ]:
Y_hat_test_final = logistic_regression_model.predict(X_test_final)


In [ ]:
Y_hat_test_final
collections.Counter(Y_hat_test_final)

New

In [ ]:
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from torchtext.vocab import build_vocab_from_iterator

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score
import seaborn as sns
from matplotlib import pyplot as plt
import regex as re
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import spacy


In [ ]:
v=Vocab(["a","b"])

In [ ]:
v[0]

In [ ]:
train_file = pd.read_csv('../data//train.csv')
final_test_file = pd.read_csv('../data/test.csv')

In [ ]:
train_file.head(5)

In [ ]:
print(np.where(pd.isnull(train_file)))
print(np.where(pd.isnull(final_test_file)))

In [ ]:
main_train_labels= train_file['label']
train_file['combined_titles']= train_file['title1_en']+' '+train_file['title2_en']

final_test_file['combined_titles']=final_test_file['title1_en']+' '+train_file['title2_en']

In [ ]:
train_split, validation_split = train_test_split(train_file, test_size=0.3,stratify=train_file['label'],random_state=9)

In [ ]:
print(Counter(train_split['label']))
print(Counter(test_split['label']))

In [ ]:
train_split['combined_titles']


In [ ]:
def preprocessing(txt):
    txt=re.sub('[^a-zA-Z0-9 ]','',txt)
    txt=txt.lower()
    return txt

main_train_titles=train_split['combined_titles'].apply(preprocessing)
main_validation_titles=validation_split['combined_titles'].apply(preprocessing)
main_test_titles=final_test_file['combined_titles'].apply(preprocessing)

In [ ]:
label_dict={"unrelated":0,"agreed":1,"disagreed":-1}
def label_encoding(txt):
    return label_dict[txt]

main_train_labels=train_split['label'].apply(label_encoding)
main_validation_labels=validation_split['label'].apply(label_encoding)

In [ ]:
# from spacy.lang.en import English
# en=English()
tokenizer=get_tokenizer('spacy',language="en_core_web_sm")
def tokenize_titles(titles,tokenizer):
    counter=Counter()
    for text in titles:
        counter.update(tokenizer(text))
    return Vocab(counter),counter

ee,ff=tokenize_titles(main_train_titles.to_list(),spacy_tokenizer)

In [ ]:
len(ff.keys())

In [ ]:
torch_tokenizer=get_tokenizer("basic_english")
title_tokens=

In [ ]:
ee['glass']#,'glass', 'of', 'glass']

In [ ]:
ee

In [ ]:
ee.__getitem__('acrobat')